In [2]:
import pandas as pd
import numpy as np
import sklearn
import seaborn
import pickle

In [3]:
dfPhenotypes = pd.read_csv("sc1_Phase2_GE_Phenotype.tsv",sep="\t")
dfPhenotypes = dfPhenotypes.replace(r'^\s*$', np.nan, regex=True)
dfPhenotypes = dfPhenotypes.replace(r'^IV', 4, regex=True)
dfPhenotypes = dfPhenotypes.replace(r'^III', 3, regex=True)
dfPhenotypes = dfPhenotypes.replace(r'^II', 2, regex=True)
dfPhenotypes = dfPhenotypes.replace(r'^I', 1, regex=True)
dfPhenotypes['WHO_GRADING'].replace(np.nan, 0, regex=True,inplace=True)
dfPhenotypes.head()

,PATIENTID,SEX,RACE,WHO_GRADING,CANCER_TYPE
0,patient_54,NaN,NaN,2.0,OLIGODENDROGLIOMA
1,patient_456,NaN,NaN,0.0,OLIGODENDROGLIOMA
2,patient_484,NaN,NaN,0.0,GBM
3,patient_441,NaN,NaN,0.0,GBM
4,patient_370,NaN,NaN,0.0,GBM


In [4]:
dfFeatures = pd.read_csv("sc1_Phase2_GE_FeatureMatrix.tsv",sep="\t")
dfFeatures.head()

,PATIENTID,A1BG,A1BG.AS1,A1CF,A2M,A2M.AS1,A2ML1,A4GALT,A4GNT,AA06,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,patient_54,6.552369,6.516572,5.670704,12.696699,6.244980,5.494503,7.031448,6.186457,6.245582,...,6.529301,7.635329,6.345017,5.998962,7.453650,6.016575,8.809521,9.755064,8.466902,8.261902
1,patient_456,6.341531,6.403964,5.575675,11.667261,6.208729,5.516190,7.077214,6.096601,6.406412,...,6.842374,8.466620,6.639999,6.318365,7.735993,6.054752,9.034913,9.474861,8.680185,8.380745
2,patient_484,6.150328,6.452206,5.465605,12.384793,5.971820,5.339407,6.787843,6.217803,6.311967,...,8.086970,9.419296,6.143196,5.972197,7.710260,6.034905,9.327712,11.646340,8.511510,8.710605
3,patient_441,6.569596,6.428329,5.404905,12.918034,5.915066,5.295160,6.975057,6.054217,6.471112,...,8.308862,9.821432,6.300042,6.099162,7.749354,6.056110,9.418540,12.472090,8.651621,8.839366
4,patient_370,6.902578,6.343173,5.243564,13.215173,6.058236,5.203713,6.786196,6.035803,6.118639,...,7.746663,9.732796,6.336464,5.987494,7.444628,5.878318,9.493086,10.977615,8.479319,8.829912


In [5]:
dfMerged = pd.merge(dfPhenotypes,dfFeatures,how="inner",on="PATIENTID")
dfMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Columns: 19340 entries, PATIENTID to ZZZ3
dtypes: float64(19336), object(4)
memory usage: 14.2+ MB


In [6]:
featuresFile = "sc1_phase1_features.txt"
with open(featuresFile) as f:
    featuresList = f.read().split(',')
print(featuresList)

['CHI3L1', 'IGFBP2', 'LTF', 'COL3A1', 'PTX3', 'EMP3', 'ADM', 'POSTN', 'TIMP1', 'ANXA1', 'MEOX2', 'NNMT', 'ETNPPL', 'KLRC3', 'CTHRC1', 'CNTN3', 'PDPN', 'RBP1', 'VEGFA', 'LAMB1', 'MCUB', 'GPR17', 'LUM', 'SCG3', 'CHST9', 'SOD2', 'KCND2', 'CHI3L2', 'COL4A2', 'TOX3', 'TMEM100', 'SNAP91', 'FLJ16779', 'MOXD1', 'SLC14A1', 'SLC2A10', 'FCGBP', 'CSMD3', 'COL6A3', 'RAB34', 'SH3GL2', 'S100A8', 'FERMT1', 'LOXL1', 'SUSD5', 'IGFBP3', 'CD163', 'DIRAS3', 'BMP2', 'CHGB', 'DCX', 'LGALS3', 'GBP1', 'AKR1C3', 'SLC16A3', 'PLPPR1', 'CFI', 'SERPINE1', 'C1R', 'LOX', 'GDF15', 'CXCL8', 'NOG', 'TNFAIP6', 'UG0898H09', 'LINC00844', 'METTL7B', 'SFRP2', 'MYL9', 'CCL2', 'PHACTR3', 'COL5A2', 'IL13RA2', 'BCAT1', 'DPP10', 'GALNT13', 'MGP', 'GSAP', 'SMOC1', 'CAV2', 'LOC389906', 'EYA1', 'PLAT', 'SERPINA3', 'F13A1', 'GPNMB', 'ARHGAP36', 'DLL3', 'CSDC2', 'INA', 'ANXA2', 'B3GAT2', 'PLIN2', 'OCIAD2', 'NSG2', 'SERPING1', 'CA10', 'FAM155A', 'FUT9', 'NKAIN4', 'KCNN2', 'PHLDA2', 'FREM3', 'ESM1', 'H19', 'LINC01088', 'TGFBI', 'IGF2BP3

In [7]:
x_test = dfMerged[featuresList]

In [8]:
finalFile = "..\..\phase1\sc1_phase1\sc1_phase1_model.sav"
loaded_model = pickle.load(open(finalFile, 'rb'))
predictions = loaded_model.predict(x_test)
print(predictions)


[1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1]
